In [1]:
import sqlite3
import pandas as pd

In [9]:
def importdb(db):
     conn = sqlite3.connect(db)
     c = conn.cursor()
     c.execute("SELECT name FROM sqlite_master WHERE type='table';")
        
     for table in c.fetchall():
         yield list(c.execute('SELECT * from ?;', (table[0],)))

In [3]:
db = "NETSDB.db"

In [10]:
conn = sqlite3.connect(db,charset='utf8')

TypeError: 'charset' is an invalid keyword argument for this function

In [5]:
c = conn.cursor()

NameError: name 'conn' is not defined

In [5]:
a = c.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [29]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('MOVESDC',), ('NETSDC',), ('NAICSDC',)]


In [30]:
c.MOVESDC

AttributeError: 'sqlite3.Cursor' object has no attribute 'MOVESDC'

In [31]:
movesdc = c.execute("SELECT * FROM MOVESDC")

In [34]:
movesdc.head()

AttributeError: 'sqlite3.Cursor' object has no attribute 'head'

In [2]:
#df = pd.read_sql_query("select * from MOVESDC limit 5;", conn)
df = pd.read_sql_query("select * from NETSDC;", conn)


NameError: name 'conn' is not defined

In [35]:
df.head()

,DunsNumber,NAICS90,NAICS91,NAICS92,NAICS93,NAICS94,NAICS95,NAICS96,NAICS97,NAICS98,...,NAICS06,NAICS07,NAICS08,NAICS09,NAICS10,NAICS11,NAICS12,NAICS13,NAICS14,NAICS15
0,002104072,,,,,,,,,,...,,,,,448150,448150,448150,448150,448150,448150
1,002105550,,,,,,,,,,...,,,,561990,,,,,,
2,002106325,,,,,,,,,,...,,,,,,811192,811192,811192,,
3,002106668,,,,,,,,,,...,,,,,561990,561990,561990,561990,561990,561990
4,002109400,,,,,,,,,,...,,,,,,624410,624410,624410,624410,624410


In [10]:
corr_matrix = df.corr()

In [26]:
df['MoveYear'].value_counts()

2009    14007
2011    13792
2010    13679
2012    13189
2004    12478
2003    12458
2008    12046
2013    12033
2014    11779
2001    10476
2006    10362
2007     9887
2005     9680
2002     9469
2000     8581
1997     7854
1998     7637
1995     7556
1996     7534
1999     7280
1990     7174
1993     6955
1991     6847
1994     6281
1992     5902
Name: MoveYear, dtype: int64

In [20]:
cm = df.corr(method='pearson')

In [21]:
import seaborn as sns
corr = cm
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

ValueError: zero-size array to reduction operation minimum which has no identity

In [22]:
cm

""


In [31]:
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()


uniform_data = np.random.rand(10, 12)
ax = sns.heatmap(df)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [29]:
flights = sns.load_dataset("flights")

# CSV Work

In [2]:
import os.path
#C:\Users\mwatson\Documents\SharePoint\The DC Policy Center - Documents\Data\NETS Data\NETS Subsets
data_folder = os.path.join("C:\\", "Users","mwatson","Documents","SharePoint","The DC Policy Center - Documents","Data","Nets Data","NETS Subsets")

file_to_open = os.path.join(data_folder, "NETS2015_general.csv")


df = pd.read_csv(file_to_open,encoding="ISO-8859-1")

In [4]:
df['Longitude'].head()

0    77.0715
1    78.1462
2    77.0549
3    77.0200
4    77.3147
Name: Longitude, dtype: float64

In [12]:
df.keys()

Index(['DunsNumber', 'Company', 'TradeName', 'City', 'State', 'ZipCode',
       'ZIP4', 'Region', 'HQDuns', 'HQCompany', 'HQTradeName', 'HQCity',
       'HQState', 'HQZipCode', 'HQZIP4', 'Subsidiary', 'Related', 'Kids',
       'CBSA', 'FipsCounty', 'CityCode', 'Latitude', 'Longitude', 'LevelCode',
       'Industry', 'IndustryGroup', 'OutofBis', 'YearStart', 'MoveYears',
       'LastMove', 'MoveSIC4', 'OriginCity', 'OriginState', 'OriginZIP',
       'DestCity', 'DestState', 'DestZIP', 'MoveEmp', 'EmpC', 'MoveSales',
       'MoveSalesC', 'PubPriv', 'LegalStat', 'ForeignOwn', 'ImpExpInd',
       'GovtContra', 'Minority', 'GenderCEO', 'WomenOwned', 'Relocate',
       'MoveOften', 'Cottage', 'FirstYear', 'LastYear', 'City_First',
       'State_First', 'ZipCode_First', 'CBSA_First', 'FipsCounty_First',
       'CityCode_First'],
      dtype='object')

In [13]:
lat = df['Latitude'][0]
lon = df['Longitude'][0]
print(lon)

77.0715


In [16]:
import censusgeocode as cg # https://github.com/fitnr/censusgeocode

In [54]:
ret = cg.coordinates(x=-lon, y=lat)

In [55]:
ret['Census Tracts'][0]['TRACT']

'705200'

In [16]:
df['lat_lon_to_tract'] = ''


In [14]:
def get_tract(lat,lon):
    try:
        ret = cg.coordinates(x=-lon, y=lat)
    except:
        print('failed')
    #print(ret)
    try:
        tract = int(ret['Census Tracts'][0]['TRACT'])/100
        tract = %.2f
    except:
        tract=0
    #print(tract)
    return(tract)

In [17]:
i = 0
tract_list = []
for i in range(20):
    lat = df['Latitude'][i]
    lon = df['Longitude'][i]
    tract = get_tract(lat,lon)
    tract_list.append(tract)
    
    df['lat_lon_to_tract'][i] = tract
    i+=1

failed
failed
failed
failed
failed
failed
failed


C:\Users\mwatson\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed


In [ ]:
i = 20
tract_list = []
for i in range(40):
    lat = df['Latitude'][i]
    lon = df['Longitude'][i]
    tract = get_tract(lat,lon)
    tract_list.append(tract)
    
    df['lat_lon_to_tract'][i] = tract
    i+=1

In [18]:
df['lat_lon_to_tract'].head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: lat_lon_to_tract, dtype: object

In [101]:
len(df) / 10

190344.3

In [99]:

getcontext().prec = 1
t = Decimal(30.1)
print(t)

30.10000000000000142108547152020037174224853515625


In [77]:
l = 100

50/l * 100

50.0

In [102]:
file_to_open = os.path.join(data_folder, "NETS2015_general_with_tract.csv")
dff = pd.read_csv(file_to_open,encoding="ISO-8859-1")

In [105]:
dff.lat_lon_to_tract.value_counts(0)

960400.0    24
0.0         19
500.0       18
970400.0    15
810901.0    12
610505.0    10
507602.0     9
930201.0     9
810800.0     8
610400.0     8
610300.0     7
10700.0      7
611014.0     7
611004.0     7
970702.0     7
960900.0     6
611006.0     6
20304.0      6
504201.0     6
10103.0      6
811000.0     6
610602.0     6
950200.0     6
930400.0     6
930502.0     6
506200.0     5
970804.0     5
610604.0     5
508101.0     5
10212.0      5
            ..
496100.0     1
802804.0     1
802702.0     1
471500.0     1
470600.0     1
431400.0     1
451700.0     1
431500.0     1
432202.0     1
432702.0     1
803524.0     1
803516.0     1
440900.0     1
450702.0     1
451100.0     1
451200.0     1
451500.0     1
451800.0     1
462201.0     1
452000.0     1
452100.0     1
452101.0     1
452102.0     1
972702.0     1
452600.0     1
457100.0     1
460800.0     1
461201.0     1
461602.0     1
980100.0     1
Name: lat_lon_to_tract, dtype: int64

In [83]:
%%bash 
echo head failed.txt

head failed.txt


In [19]:
import multiprocessing as mp
output = mp.Queue()


In [ ]:
def cube(x):
    return x**3


In [ ]:
pool = mp.Pool(processes=4)
results = [pool.apply(cube, args=(x,)) for x in range(1,7)]
print(results)


In [38]:
import requests
import grequests

In [24]:
lat = [38.9758,37.9758,38.8758,38.9758,38.9558]
lon = [77.0715

url = "https://geo.fcc.gov/api/census/block/find?latitude={}&longitude=-{}&showall=false&format=json".format(str(lat),str(lon))

resp = requests.get(url)

In [27]:
fips = resp.json()['Block']['FIPS']

In [28]:
fips[5:10]

'70520'

In [75]:
lat_list = []
lon_list = []
for i in range(20):
    lat = df['Latitude'][i]
    lon = df['Longitude'][i]
    lat_list.append(lat)
    lon_list.append(lon)
lat = df['Latitude'][len(df['Latitude']) + 1 ] = 0
lon = df['Longitude'][len(df['Latitude']) + 1 ] =0 
lat_list.append(lat)
lon_list.append(lon)

In [76]:
url_list = []
for i in range(len(lat_list)):
    lat = lat_list[i]
    lon = lon_list[i]
    url = "https://geo.fcc.gov/api/census/block/find?latitude={}&longitude=-{}&showall=false&format=json".format(str(lat),str(lon))
    url_list.append(url)

In [77]:
rs = [grequests.get(u) for u in url_list]

In [78]:
grequests.map(rs)

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

In [80]:
rs[20].response.json()['Block']['FIPS'] == None


True

In [74]:
print(url_list)

['https://geo.fcc.gov/api/census/block/find?latitude=38.9758&longitude=-77.0715&showall=false&format=json', 'https://geo.fcc.gov/api/census/block/find?latitude=39.197&longitude=-78.1462&showall=false&format=json', 'https://geo.fcc.gov/api/census/block/find?latitude=38.8975&longitude=-77.0549&showall=false&format=json', 'https://geo.fcc.gov/api/census/block/find?latitude=38.8948&longitude=-77.02&showall=false&format=json', 'https://geo.fcc.gov/api/census/block/find?latitude=38.7508&longitude=-77.3147&showall=false&format=json', 'https://geo.fcc.gov/api/census/block/find?latitude=40.2709&longitude=-77.6845&showall=false&format=json', 'https://geo.fcc.gov/api/census/block/find?latitude=42.1472&longitude=-71.2427&showall=false&format=json', 'https://geo.fcc.gov/api/census/block/find?latitude=42.1367&longitude=-71.2659&showall=false&format=json', 'https://geo.fcc.gov/api/census/block/find?latitude=38.7262&longitude=-77.0945&showall=false&format=json', 'https://geo.fcc.gov/api/census/block/f

In [82]:
for item in range(len(rs)):
    print(rs[item].response.json()['Block']['FIPS'])

240317052003026
510690511022038
110010056002005
110010058002010
510594922011000
420550101001000
250214111003003
250214113022000
510594160001000
250173171022042
245100104001008
240317008173004
390490062303009
510594525012024
240253038031017
240338070004030
240338005171032
510594711004005
511579501001132
240054303002001
None
